In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('balanced_df.csv')
df.head(-10)

,reviewer_id,store_location,latitude,longitude,date,month,year,title,review,review-label
0,151782.0,US,37.09024,-95.712891,2021,6,2008 00:00:00,I like the tee shirt I ordered butï¿½ï¿½ï¿½ï¿½...,like tee shirt order truth qualiti shirt self ...,1.0
1,27155.0,US,37.09024,-95.712891,2023,1,2001 00:00:00,Poor quality and smelled bad,order two differ item differ vendor shirt chea...,1.0
2,72707.0,US,37.09024,-95.712891,2022,8,2007 00:00:00,I did not get my complete order,get complet order im miss mask,1.0
3,2766.0,US,37.09024,-95.712891,2023,6,2010 00:00:00,I never received the shirt I orderedï¿½ï¿½ï¿½,never receiv shirt order get touch anyon,1.0
4,240808.0,US,37.09024,-95.712891,2019,9,2028 00:00:00,"Too small, poor quality",normal xl order xl could put long sleev shirt ...,1.0
...,...,...,...,...,...,...,...,...,...,...
14899,155659.0,US,37.09024,-95.712891,2021,4,2025 00:00:00,Not a wise choice,tee shirt receiv small larg small wear first t...,2.0
14900,240408.0,US,37.09024,-95.712891,2019,10,2001 00:00:00,I only buy 50/50 cotton blends to avoidï¿½ï¿½ï¿½,buy cotton blend avoid shrinkag pre inquir mat...,2.0
14901,156969.0,US,37.09024,-95.712891,2021,4,2013 00:00:00,The print on the tee shirt is nice,print tee shirt nice size way small return,2.0
14902,27740.0,US,37.09024,-95.712891,2022,12,2024 00:00:00,I ordered a Buccaneers t-shirt for aï¿½ï¿½ï¿½,order buccan shirt christma present shirt mi s...,2.0


In [3]:
X = list(df['review'])
Y = df['review-label']

In [4]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

In [5]:
x_train, x_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
x_train, x_val, Y_train, Y_val = train_test_split(x_train, Y_train, test_size=0.25, random_state=42)


In [6]:
x_train = [str(x) for x in x_train if isinstance(x, (str, int, float))]
x_val = [str(x) for x in x_val if isinstance(x, (str, int, float))]
x_test = [str(x) for x in x_test if isinstance(x, (str, int, float))]
#

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def tokenize_data(texts, tokenizer, max_length=64):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

In [9]:
train_encodings = tokenize_data(x_train, tokenizer)
val_encodings = tokenize_data(x_val, tokenizer)
test_encodings = tokenize_data(x_test, tokenizer)

In [10]:
train_encodings['input_ids'][602].shape


TensorShape([64])

In [11]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   # Or 'val_accuracy' if you prefer
    patience=3,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best performance
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the checkpoint path in your Google Drive
checkpoint_path = "/content/drive/MyDrive/model_checkpoints/epoch_{epoch:02d}.h5"

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,   # Save model here
    save_weights_only=False,     # Save only weights, not the entire model architecture
    monitor='val_loss',         # Monitor validation loss to determine when to save
    save_best_only=False,       # Save the model after every epoch
    verbose=1,                  # Print a message when saving the model
    period=5                    # Save after every 5 epochs
)

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
history = model.fit(
    train_encodings['input_ids'], np.array(Y_train),
    validation_data=(val_encodings['input_ids'], np.array(Y_val)),
    epochs=10,
    batch_size=32,
    callbacks=[checkpoint_callback, early_stopping]
)

Epoch 1/10


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
280/280 [==============================] - 710s 2s/step - loss: 0.6024 - accuracy: 0.7212 - val_loss: 0.5603 - val_accuracy: 0.7375
Epoch 2/10
280/280 [==============================] - 630s 2s/step - loss: 0.5273 - accuracy: 0.7385 - val_loss: 0.5778 - val_accuracy: 0.7389
Epoch 3/10
280/280 [==============================] - 634s 2s/step - loss: 0.4787 - accuracy: 0.7729 - val_loss: 0.5404 - val_accuracy: 0.7318
Epoch 4/10
280/280 [==============================] - 628s 2s/step - loss: 0.4030 - accuracy: 0.8215 - val_loss: 0.6295 - val_accuracy: 0.7305
Epoch 5/10
280/280 [==============================] - 626s 2s/step - loss: 0.2987 - accuracy: 0.8738 - val_loss: 0.7677 - val_accuracy: 0.7090
Epoch 6/10
280/280 [==============================] - 624s 2s/step - loss: 0.1985 - accuracy: 0.9240 - val_loss: 0.9647 - val_accuracy: 0.7067


In [15]:
# Evaluate the model on validation set
results = model.evaluate(val_encodings['input_ids'], np.array(Y_val))
print(f"Validation Accuracy: {results[1]:.4f}")

94/94 [==============================] - 63s 672ms/step - loss: 0.5404 - accuracy: 0.7318
Validation Accuracy: 0.7318


In [ ]:
test_prediction = model.predict(val_encodings['input_ids'])

In [ ]:
# Save the model at the end of training
final_model_path = "/content/drive/MyDrive/model_checkpoints/final_model.h5"
model.save(final_model_path)
